# Keras Pytorch etc.

In [3]:
import numpy as np
import tensorflow as tf

from numpy import array
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

In [50]:
# A Simple Network with One Node weights

model = tf.keras.Sequential()
model.add(Dense(1, activation='relu', input_dim=1))
opt=tf.keras.optimizers.SGD(learning_rate=.001, momentum=0.0, nesterov=False)
#model.compile(optimizer=opt, loss='binary_crossentropy')
model.compile(optimizer=opt, loss='mse')

# ~~~ Set custom weights
wts1=( np.array([[0.5]]),  np.array([0]) )
print('Weights(1):' , wts1[0].shape, wts1[0] )
model.layers[0].set_weights(wts1)
wts11=model.layers[0].get_weights()
print('Weights:' , wts11[0].shape, wts11[1].shape, wts11[0], wts11[1] )

#--- Do one pass through NN
data=np.array([[1]]).reshape(1,-1)
y = np.array([10])

print('Call Predict on untrained model'  )
yhat = model.predict(data, verbose=0)
print("Before Fit: " , yhat)

for i in range(5):
    f = model.fit(data, y, epochs=1000, verbose=0)
    yhat = model.predict(data, verbose=0)
    print(f"Fit {i}: {data} {y} {yhat}" )
    #print(f"1/1 - f.loss\r")
print(f"#--- Weights after {i} pass");
wts1=model.layers[0].get_weights()
op=model.layers[0].output

yhat = model.predict(data, verbose=0)
wts11=model.layers[0].get_weights()
print('Weights:' , wts11[0].shape, wts11[1].shape, wts11[0], wts11[1],  "Output: ", yhat )



Weights(1): (1, 1) [[0.5]]
Weights: (1, 1) (1,) [[0.5]] [0.]
Call Predict on untrained model
Before Fit:  [[0.5]]
Fit 0: [[1]] [10] [[9.827402]]
Fit 1: [[1]] [10] [[9.996862]]
Fit 2: [[1]] [10] [[9.999882]]
Fit 3: [[1]] [10] [[9.999882]]
Fit 4: [[1]] [10] [[9.999882]]
#--- Weights after 4 pass
Weights: (1, 1) (1,) [[5.249941]] [4.7499404] Output:  [[9.999882]]


In [277]:
print(op)
#oo=tf.compat.v1.keras.backend.get_session(op_input_list=())
#op.eval(session=K.get_session())

Tensor("dense_22_1/Identity:0", shape=(None, 1), dtype=float32)


In [33]:
# A Simple Network with predefined weights

model = tf.keras.Sequential()
model.add(Dense(2, activation='relu', input_dim=2))
model.add(Dense(1))
opt=tf.keras.optimizers.SGD(learning_rate=0.5, momentum=0.0, nesterov=False)
model.compile(optimizer=opt, loss='binary_crossentropy')

# ~~~ Set custom weights
wts0=( np.array([[0.5, 0.1], [0.1, 0.5]]),  np.array([0,0]) )
print('Weights(0):' , wts0[0].shape, wts0[1].shape, "\n", wts0[0], wts0[1] )

model.layers[0].set_weights(wts0)
wts01=model.layers[0].get_weights()
print('Weights:' , wts01[0].shape, wts01[1].shape, wts01[0], wts01[1] )

wts1=( np.array([[0.5], [0.5]]),  np.array([0]) )
print('Weights(1):' , wts1[0].shape, wts1[1].shape, "\n", wts1[0], wts1[1] )
model.layers[1].set_weights(wts1)
wts11=model.layers[1].get_weights()
print('Weights:' , wts11[0].shape, wts11[1].shape, wts11[0], wts11[1] )

#--- Do one pass through NN
data=np.array([[1,1]])
y =np.array([10])
f=model.fit(data, y, epochs=1, verbose=1)

print("#--- Weights after one pass");
wts1=model.layers[0].get_weights()
print('Weights:' , wts1[0].shape, wts1[1].shape, wts1[0], wts1[1] )
wts11=model.layers[1].get_weights()
print('Weights:' , wts11[0].shape, wts11[1].shape, wts11[0], wts11[1] )


Weights(0): (2, 2) (2,) 
 [[0.5 0.1]
 [0.1 0.5]] [0 0]
Weights: (2, 2) (2,) [[0.5 0.1]
 [0.1 0.5]] [0. 0.]
Weights(1): (2, 1) (1,) 
 [[0.5]
 [0.5]] [0]
Weights: (2, 1) (1,) [[0.5]
 [0.5]] [0.]
1/1 [==============================] - 0s 193ms/step - loss: -3.1384
#--- Weights after one pass
Weights: (2, 2) (2,) [[10.291664   9.8916645]
 [ 9.8916645 10.291664 ]] [9.791664 9.791664]
Weights: (2, 1) (1,) [[12.249997]
 [12.249997]] [19.583328]


In [213]:
model.layers[1].output


<tf.Tensor 'dense_3_1/Identity:0' shape=(None, 1) dtype=float32>

In [54]:
dataset1 = array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
dataset2 = array([10, 20, 30, 40, 50, 60, 70, 80, 90, 100])
n_input = 2
generator = TimeseriesGenerator(dataset1, dataset2, length=n_input, batch_size=3)
print(f'Samples: {len(generator)}, Shape:{generator[0][0].shape}')
for i in range(len(generator)):
    x, y = generator[i]
    print(f'{x} => {y}')

Samples: 3, Shape:(3, 2)
[[1 2]
 [2 3]
 [3 4]] => [30 40 50]
[[4 5]
 [5 6]
 [6 7]] => [60 70 80]
[[7 8]
 [8 9]] => [ 90 100]


In [56]:
generator = TimeseriesGenerator(dataset1, dataset2, length=n_input, batch_size=8)

model = Sequential()
model.add(Dense(5, activation='relu', input_dim=n_input))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')
model.fit_generator(generator, steps_per_epoch=1, epochs=1000, verbose=0)
x_input = array([3, 4]).reshape((1, n_input))
yhat = model.predict(x_input, verbose=0)
print(yhat)
for i in range (5):
    model.fit_generator(generator, steps_per_epoch=1, epochs=1000, verbose=0)
    x_input = array([3, 4]).reshape((1, n_input))
    yhat = model.predict(x_input, verbose=0)
    print(yhat)

[[0.98738676]]


In [143]:
# A Simple Network with predefined weights

model = Sequential()
model.add(Dense(2, activation='relu', input_dim=2))
model.add(Dense(1))
model.compile(optimizer='sgd', loss='mse')

# ~~~ Set custom weights
wts0=( np.array([[0.5, 0.1], [0.1, 0.2]]),  np.array([0,0]) )
print('Weights(0):' , wts0[0].shape, wts0[1].shape, "\n", wts0[0], wts0[1] )

model.layers[0].set_weights(wts0)
wts01=model.layers[0].get_weights()
print('Weights:' , wts01[0].shape, wts01[1].shape, wts01[0], wts01[1] )

wts1=( np.array([[0.5], [0.5]]),  np.array([0]) )
print('Weights(1):' , wts1[0].shape, wts1[1].shape, "\n", wts1[0], wts1[1] )
model.layers[1].set_weights(wts1)
wts11=model.layers[1].get_weights()
print('Weights:' , wts11[0].shape, wts11[1].shape, wts11[0], wts11[1] )

#--- Do one pass through NN
data=np.array([[1,1]])
model.fit(data, [1], epochs=1, verbose=1)

wts1=model.layers[0].get_weights()
print('Weights:' , wts1[0].shape, wts1[1].shape, wts1[0], wts1[1] )


Weights(0): (2, 2) (2,) 
 [[0.5 0.1]
 [0.1 0.2]] [0 0]
Weights: (2, 2) (2,) [[0.5 0.1]
 [0.1 0.2]] [0. 0.]
Weights(1): (2, 1) (1,) 
 [[0.5]
 [0.5]] [0]
Weights: (2, 1) (1,) [[0.5]
 [0.5]] [0.]
Epoch 1/1
1/1 [==============================] - 0s 32ms/step - loss: 0.3025
Weights: (2, 2) (2,) [[0.5055 0.1055]
 [0.1055 0.2055]] [0.0055 0.0055]


In [126]:
wts01=model.layers[1].get_weights()
print('Weights:' , wts01[0].shape, wts01[1].shape, wts01[0], wts01[1] )


Weights: (2, 1) (1,) [[ 0.43578127]
 [-1.075966  ]] [0.02141919]
